# Ebay Car Sales

In this project I will work through the dataset of used cars from the classified section of the german ebay website. The dataset is available at the [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data).

The data file contains 371+ thousands of car listings. Although, for the purpose of faster analysis, the subset of data, which contains 50 thousand records, is obtained from the [Dataquest](dataquest.io), the same analysis precedure can be carried out on the whole dataset with minor modifications.

The data dictionary provided with the dataset is as follows:

1. **`dateCrawled`** - When this (car) ad was first crawled. All field-values are taken from this data.
2. **`name`** - Name of the car.
3. **`seller`** - Whether the seller is private or a dealer.
4. **`offerType`** - The type of listing.
5. **`price`** - The price on the ad to sell the car.
6. **`abtest`** - Whether the listing is included in an A/B test.
7. **`vehicleType`** - The type of vehicle.
8. **`yearOfRegistration`** - The year in which the car was first registered.
9. **`gearbox`** - The transmission type.
10. **`powerPS`** - The power of the car in PS.
11. **`model`** - The car model name.
12. **`kilometer`** - How many kilometers the car has driven.
13. **`monthOfRegistration`** - The month in which the car was first registered.
14. **`fuelType`** - What type of fuel car uses.
15. **`brand`** - The brand of the car.
16. **`notRepairedDamange`** - If the car has the damage which is not yet repaired.
17. **`dateCreated`** - The date on which the listing was created.
18. **`nrOfPictures`** - The number of pictures in the ad.
19. **`postalCode`** - The postal code for the location of the vehicle.
20. **`lastSeenOnline`** - When the crawler saw this ad last online.

**The aim of this project is to clean the data and analyze the used car listings.**

In [1]:
import pandas as pd
import numpy as np
import re
from collections import OrderedDict

In [2]:
autos = pd.read_csv("autos.csv", encoding='Latin-1')

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


While there are 20 columns in total, 15 of which are type object(string). Each column having 50,000 records, there are some columns which are missing some recordings.

Column label and associated approximate missings value percentage:
- `vehicleType` - 10%
- `gearBox` - 6%
- `model` - 6%
- `fuelType` - 10%
- `notRepairedDamage` - 20%

It seems the **`notRepairedDamage`** column is missing a lot of values.

Except for these columns with missing values, all other features(columns) at the moment seem to be available for each car listing. It is possible that we may be surprised to see in other columns non-sensible values later on.

In [5]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

**Let's convert the column names from camelcase to snakecase and for some columns let us give new more meaningful names based on the data dictionary.**

In [6]:
old_col_names = autos.columns
new_col_names = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']
autos.columns = new_col_names

Okay, so I have made some changes to the column names as can be seen in the above cell. All the longer and less meaningful names are changed to be shorter and more meaningful.

While camelCase naming convention is not bad, I have taken the pleasure and **converted all column names from camelCase to snake_case** which is (for me) much faster and easier to work with throughout the analysis process.

- **`yearOfRegistration`**  ==>  **`registration_year`**
- **`monthOfRegistration`**  ==>  **`registration_month`**
- **`notRepairedDamange`**  ==> **`unrepaired_damage`**
- **`dateCreated`**  ==>  **`ad_created`**

In [7]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


**[NOTES_FROM_COURSE]**

Initially we will look for: - **Text columns where all or almost all values are the same.** These can often be dropped as they don't have useful information for analysis.

The following methods are helpful for exploring the data: - DataFrame.describe() (with include='all' to get both categorical and numeric columns) - Series.value_counts() and Series.head() if any columns need a closer look.

In [8]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-21 20:37:19,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [9]:
autos['unrepaired_damage'].value_counts(dropna=False)

nein    35232
NaN      9829
ja       4939
Name: unrepaired_damage, dtype: int64

- The **`date_crawled`, `registration_year`,  `registration_month`, `ad_created` and `last_seen`** features are a string dtype columns. We can parse the date out of the these feature columns for each car listing and then convert these feature column's dtype to be datetime.

- As can be seen from the dataset description, there are two columns - **'seller'** & **'offer_type'** - which has two unique values each. Since almost all(49999) car listings have same value - value being **`private`** - for the **`seller`** feature we can drop that feature from the dataset. Same goes for the **`offer_type`** feature.

- **`price`** column is a string dtype at the moment, which needs to be type casted to be a float.
- **`odometer`** feature column needs dtype casted to be an float column. The 'km' string needs to stripped from each row.
- As for the **`unrepaired_damage`** column, the `ja` and `nein`(German translation of Yes and No respectively) values needs to be replaced by `yes` and `no` strings or equivalent 1 and 0 numbers. Also that since there is no mention of unrepaired damange for some car listings, we'll assume that these cars did not have any unrepaired damage and hence not recorded. ie. NaN values will be treated as a `No` value for this feature column.
- **`nr_of_pictures`** feature column, which stands for the number of pictures, has same value - 0(zero) - throughout the whole column. Hence by assuming it contributes no new value to the dataset we'll drop that feature column.
- I'm not sure if **`postal_code`** feature column contribute more value to the dataset by being `float` dtype than it will by being `str` dtype. So for the moment, I will keep it as a `float` but if situation demands might convert it to `str` dtype later on.

### Columns to be dropped from the dataset:
- **`seller`**
- **`offer_type`**
- **`nr_of_pictures`**

### Columns that require data type casting:
- **`date_crawled` :** `string` to `datetime`
- **`registration_year` :** `string` to `datetime`
- **`registration_month` :** `string` to `datetime`
- **`ad_created` :** `string` to `datetime`
- **`last_seen` :** `string` to `datetime`
- **`price` :** `string` to `float`.
- **`odometer` :** `string` to `float`
- **`postal_code` :** (DECIDED TO BE LATER ON) (`float` to `str`)

### Columns that require value's rewording:
- **`unrepaired_damage`** convert nein to NO and ja to yes
- **`name`** strip the brand name off of name of the car.

### Overall, as I see at the moment, 13 feature columns require some sort of modification to them while 7 columns do not.

In [10]:
# Lets dtype cast the `price` and `odometer` feature columns.
# The workflow for converting any non-numeric column to numeric type is
# as follows:
# 1. Explore the data.
# 2. Find any patterns
# 3. Remove non-digit characters.
# 4. Convert (type cast) to numeric dtype.
# 5. [OPTIONAL] Rename the column lable if necessary.

# STEP 1 Explore
autos[['price','odometer']].sample(10)

,price,odometer
11904,"$1,100","150,000km"
43641,"$2,349","150,000km"
402,$450,"150,000km"
6098,"$9,150","150,000km"
2789,"$4,750","150,000km"
25040,"$2,850","150,000km"
16488,"$2,500","150,000km"
43420,"$15,800","40,000km"
17528,"$1,300","100,000km"
22497,"$1,450","150,000km"


- For **`price`** column, let us strip the **`$`** sign off the front and replace **`","(punctuation mark)`** by `empty character`.
- For **`odometer`** column, I'll strip the 'km' string off of the back of the sequence and replace ","(punctuation mark) by the `empty character`

In [11]:
# STEP 2 Find patterns (found it!)
# STEP 3 Remove non-digit characters

# Regex pattern for any non-digit character
pat = re.compile(r"\D")

# Loop over both the columns - price and odometer
for col in ['price', 'odometer']:
    autos[col] = autos[col].replace(pat,"")
    # STEP 4 Convert column to Numeric dtype.
    autos[col] = autos[col].astype(float)

In [12]:
# STEP 5 Rename columns if necessary.
autos.rename(columns={'odometer':'odometer_km'}, inplace=True)

In [13]:
autos[['price','odometer_km']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
price          50000 non-null float64
odometer_km    50000 non-null float64
dtypes: float64(2)
memory usage: 781.3 KB


### Let us look more closely at the _`price`_ and _`odometer`_ (now renamed to *`odometer_km`*) columns.

In [14]:
print(autos['price'].nunique())

2357


In [15]:
autos['price'].value_counts().sort_index(ascending=False).head(30)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
194000.0      1
190000.0      1
180000.0      1
175000.0      1
169999.0      1
169000.0      1
163991.0      1
163500.0      1
155000.0      1
151990.0      1
Name: price, dtype: int64

In [16]:
(50000 - autos['price'].between(100,350000).sum())*100/50000

3.552

For the top bound of outliers in auto price data, I would set it to be 350k. The reason for doing so is that, the price is going up in relatively equally spaced amount like 1-5k until it hits the 350k mark. After that the price is 999k which is almost 3x the previous one - 350k. So I believe prices above 350k are unrealistic. And anything beyond that mark will be considered an outlier.

For the lower bound of outlier, for now, I'll just pick a number which seems fair to me and that is 100 dollars.

By doing so - setting lower and upper outlier bound to be \$100 and \$350k - we're ammounting for about 3.5% percent of data loss accountable to price column.

**price non-outlier range: \$100 to \$350,000**

In [17]:
autos = autos[autos['price'].between(100,350000)]

In [18]:
autos['odometer_km'].unique().shape

(13,)

In [19]:
autos['odometer_km'].value_counts(dropna=False)

150000.0    31212
125000.0     5037
100000.0     2101
90000.0      1733
80000.0      1412
70000.0      1214
60000.0      1153
50000.0      1009
40000.0       814
30000.0       777
5000.0        760
20000.0       757
10000.0       245
Name: odometer_km, dtype: int64

#### The odometer_km column doesn't seem to have any outliers, so I'll leave it as it is.

In [20]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,48224,48224,48224,48224,48224.000000,48224,43801,48224.000000,46019,48224.000000,45829,48224.000000,48224.000000,44345,48224,39340,48224,48224.0,48224.000000,48224
unique,46571,37162,2,1,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,38232
top,2016-04-04 16:40:33,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,76,48223,48224,NaN,24826,12558,NaN,35926,NaN,3876,NaN,NaN,29203,10271,34711,1874,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,5930.371433,NaN,NaN,2004.730964,NaN,117.677609,NaN,125919.148142,5.801634,NaN,NaN,NaN,NaN,0.0,50987.919729,NaN
std,NaN,NaN,NaN,NaN,9078.372762,NaN,NaN,87.897388,NaN,201.206304,NaN,39543.339640,3.676976,NaN,NaN,NaN,NaN,0.0,25737.119986,NaN
min,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1250.000000,NaN,NaN,1999.000000,NaN,73.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30823.000000,NaN
50%,NaN,NaN,NaN,NaN,3000.000000,NaN,NaN,2004.000000,NaN,107.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49716.000000,NaN
75%,NaN,NaN,NaN,NaN,7499.000000,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71666.750000,NaN


### Let's take a closer look at date/time related columns:
- **`date_crawled`** - added by crawler
- **`registration_year`** - from the website
- **`registration_month`:** - from the website
- **`ad_created`** - from the website
- **`last_seen`** - added by crawler

The `date_crawled` , `last_seen` and `ad_created` columns are identified as string values by pandas. Let us convert the data to numeric representation so that it can be understood quantitatively."

In [21]:
autos[['date_crawled', 'ad_created', 'last_seen']].sample(10)

,date_crawled,ad_created,last_seen
27195,2016-03-28 00:36:42,2016-03-28 00:00:00,2016-04-06 10:18:03
9210,2016-04-03 20:39:31,2016-04-03 00:00:00,2016-04-03 20:39:31
10864,2016-03-27 18:37:37,2016-03-27 00:00:00,2016-03-29 08:15:21
12168,2016-04-05 07:58:47,2016-04-05 00:00:00,2016-04-07 12:15:46
36285,2016-04-02 17:00:16,2016-04-02 00:00:00,2016-04-06 17:15:44
2585,2016-03-21 11:57:02,2016-03-21 00:00:00,2016-04-06 05:45:54
38681,2016-03-09 19:53:02,2016-03-09 00:00:00,2016-04-05 14:18:43
1376,2016-03-08 13:37:51,2016-03-08 00:00:00,2016-04-05 16:44:37
28949,2016-03-17 20:36:22,2016-03-17 00:00:00,2016-03-23 09:47:53
6160,2016-03-18 13:49:10,2016-03-18 00:00:00,2016-04-05 22:17:22


In [22]:
autos['date_crawled'].value_counts().shape

(46571,)

The above mentioned dates columns:
- date_crawled
- ad_created
- last_seen

are in a form of a full timestamp. But, we don't need them to be in such a form, because it makes distribution ananlysis harder for us. So the full timestamp, such as this: **2016-03-30 00:00:00** will be converted by stripping off the time field to a form such as this: **2016-03-30**. This type of date form is much more suitable to work with and gives more control over analysis process.

In [23]:
dates_col = ['date_crawled', 'ad_created', 'last_seen']
for col in dates_col:
    autos[col] = autos[col].str[:10]

In [24]:
autos[['date_crawled', 'ad_created', 'last_seen']].sample(10)

,date_crawled,ad_created,last_seen
38609,2016-03-08,2016-03-08,2016-04-06
38886,2016-03-27,2016-03-27,2016-04-01
29631,2016-03-26,2016-03-26,2016-04-06
25790,2016-03-29,2016-03-29,2016-04-06
4165,2016-03-19,2016-03-19,2016-04-06
541,2016-03-11,2016-03-11,2016-03-13
45889,2016-03-13,2016-03-13,2016-03-18
34288,2016-03-11,2016-03-11,2016-03-26
29536,2016-04-04,2016-04-04,2016-04-06
22236,2016-03-19,2016-03-19,2016-03-27


### Monthly distribution of car listings:

In [25]:
for col in dates_col:
    print("*"*10+col+"*"*10)
    print(autos[col].value_counts(normalize=True, dropna=False).sort_index())

**********date_crawled**********
2016-03-05    0.025361
2016-03-06    0.014039
2016-03-07    0.036061
2016-03-08    0.033179
2016-03-09    0.033013
2016-03-10    0.032287
2016-03-11    0.032598
2016-03-12    0.036911
2016-03-13    0.015677
2016-03-14    0.036662
2016-03-15    0.034319
2016-03-16    0.029467
2016-03-17    0.031499
2016-03-18    0.012898
2016-03-19    0.034734
2016-03-20    0.037803
2016-03-21    0.037201
2016-03-22    0.032888
2016-03-23    0.032287
2016-03-24    0.029446
2016-03-25    0.031499
2016-03-26    0.032308
2016-03-27    0.031126
2016-03-28    0.034962
2016-03-29    0.034112
2016-03-30    0.033738
2016-03-31    0.031851
2016-04-01    0.033697
2016-04-02    0.035605
2016-04-03    0.038611
2016-04-04    0.036538
2016-04-05    0.013064
2016-04-06    0.003173
2016-04-07    0.001389
Name: date_crawled, dtype: float64
**********ad_created**********
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015

### Yearly distribution of car listings:

In [26]:
# Let's see how car listings are distributed across months
for col in dates_col:
    print("*"*10+col+"*"*10)
    print(autos[col].str[:7].value_counts(normalize=True, dropna=False).sort_index())

**********date_crawled**********
2016-03    0.837923
2016-04    0.162077
Name: date_crawled, dtype: float64
**********ad_created**********
2015-06    0.000021
2015-08    0.000021
2015-09    0.000021
2015-11    0.000021
2015-12    0.000041
2016-01    0.000249
2016-02    0.001265
2016-03    0.837342
2016-04    0.161019
Name: ad_created, dtype: float64
**********last_seen**********
2016-03    0.42342
2016-04    0.57658
Name: last_seen, dtype: float64


As can be seen from the previous two cell's outputs which represent the distribution of multiple dates associated with the car listing. But most importantly it gives information about the distribution of time when car was listed for sell and the distribution of year in which the car was registered first.

**Almost all of the crawling took place in March of 2016** the precise number being **83%** and the rest of crawling took place in April of 2016.

The interesting fact to see here is that, **most of the cars - more than 83% - were listed for online sell in the month of March 2016** and almost all of the rest followed listing in the April of same year. So overall, almost all of the cars were listed online for sell, collectively in March & April of 2016.

Since the web crawling didn't last for too long which is just two months, all of cars' listings were sighted for the last time in those two months.


In [27]:
print(autos['registration_year'].value_counts().sort_index()[:15],"\n")
print(autos['registration_year'].value_counts().sort_index(ascending=False)[:15],"\n")

1000    1
1001    1
1111    1
1800    2
1910    2
1927    1
1929    1
1931    1
1934    2
1937    4
1938    1
1939    1
1941    2
1943    1
1948    1
Name: registration_year, dtype: int64 

9999       3
9000       1
8888       1
6200       1
5911       1
5000       3
4800       1
4500       1
4100       1
2800       1
2019       2
2018     468
2017    1383
2016    1202
2015     380
Name: registration_year, dtype: int64 



**Moving on to the registration year, **
- The minimum value is 1000, before cars were invented.
- The maximum value is 9999, many years into the future.

Because a car can not be registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. As for the accurate earlier registration year, it would be reasonable to consider it somewhere  in the first few decades of 1900s.

Since, car listings seem to have a relatively equally spaced registration year after 1927, It would be a good choice to think it as the earliest valid registration year. Prior to 1927 there're couple of listing dated back to 1910 and nowhere in between. So 1927 would make a better choice as lower bound for the registraion year.


In [28]:
# Let's remove car listings having registration year
# outside of bound 1927-2016
autos = autos[autos.registration_year.between(1927,2016)]

In [29]:
# let's plot registration year distribution
autos[['registration_year','registration_month']].describe()

,registration_year,registration_month
count,46350.000000,46350.000000
mean,2002.943797,5.847141
std,7.101102,3.661159
min,1927.000000,0.000000
25%,1999.000000,3.000000
50%,2003.000000,6.000000
75%,2008.000000,9.000000
max,2016.000000,12.000000


In [30]:
autos['registration_year'].value_counts(normalize=True).sort_values(ascending=False)

2000    0.066969
2005    0.062805
1999    0.062114
2004    0.058231
2003    0.058101
2006    0.057562
2001    0.056721
2002    0.053441
1998    0.050485
2007    0.049040
2008    0.047681
2009    0.044876
1997    0.041532
2011    0.034908
2010    0.034239
1996    0.029234
2012    0.028220
2016    0.025933
1995    0.025739
2013    0.017282
2014    0.014283
1994    0.013506
1993    0.009061
2015    0.008198
1992    0.007918
1991    0.007292
1990    0.007163
1989    0.003689
1988    0.002869
1985    0.002006
          ...   
1960    0.000475
1966    0.000475
1976    0.000453
1969    0.000410
1975    0.000388
1965    0.000367
1964    0.000259
1963    0.000173
1961    0.000129
1959    0.000129
1937    0.000086
1962    0.000086
1956    0.000086
1958    0.000086
1954    0.000043
1941    0.000043
1951    0.000043
1934    0.000043
1957    0.000043
1955    0.000043
1939    0.000022
1950    0.000022
1929    0.000022
1927    0.000022
1948    0.000022
1938    0.000022
1943    0.000022
1953    0.0000

The distribution of car registration year is centered around 2003. It is interesting to note that, the distribution seems to be densely populated around recent year. If we consider the registraion year span from 1927 to 2016 total of 90 years, then it is interesting to note that:
- **25% of vehicles' registration is spread over ~80% of registration span**(1927-1999, total of 73 years).
- **75% of vehicles' registration is spread over merely ~20% registration span** (2000-2016 total of 17 years).

It would be fair to conclude that large number of vehicles being sold are registered very recently in time, while a relatively small number of vehicles are registered earlier in time.

### Let's take a closer look at vehicle brands:

In [31]:
print(autos['brand'].value_counts(normalize=True))


volkswagen        0.211413
bmw               0.110183
opel              0.107228
mercedes_benz     0.096656
audi              0.086775
ford              0.069838
renault           0.047055
peugeot           0.029860
fiat              0.025609
seat              0.018252
skoda             0.016419
nissan            0.015340
mazda             0.015232
smart             0.014196
citroen           0.014045
toyota            0.012794
hyundai           0.010011
sonstige_autos    0.009536
volvo             0.009126
mini              0.008803
mitsubishi        0.008177
honda             0.007875
kia               0.007077
alfa_romeo        0.006667
porsche           0.006019
suzuki            0.005933
chevrolet         0.005674
chrysler          0.003517
dacia             0.002654
daihatsu          0.002503
jeep              0.002287
land_rover        0.002114
subaru            0.002114
saab              0.001661
jaguar            0.001532
daewoo            0.001489
trabant           0.001359
r

Highest number of vehicles - almost one-fourth of total - are of **volkswagen** brand.

Along with volkswagen brand, BMW, Opel, Mercedes-Benz, Audi and Ford are having a share of between 5-10% of total vehicles.

**Lada** has the least, being less than 0.1% of total number of vehicles. 

Let us aggregate pricing data on vehicle brands having more than 1% of share in total vehicles.

In [32]:
brands_share = autos['brand'].value_counts(normalize=True)
# brands having more than 1% of share in total listings.
top_brands = brands_share[brands_share>0.01].index


In [33]:
brand_pricing = OrderedDict()
for brnd in top_brands:
    brand_pricing[brnd] = autos.loc[autos['brand']==brnd,'price'].mean()

In [34]:
for brand,price in brand_pricing.items():
    print("{:>15}: {:<}".format(brand,round(price,2)))

     volkswagen: 5436.95
            bmw: 8381.68
           opel: 3006.0
  mercedes_benz: 8672.65
           audi: 9380.72
           ford: 3779.27
        renault: 2496.94
        peugeot: 3113.86
           fiat: 2836.87
           seat: 4433.42
          skoda: 6409.61
         nissan: 4756.66
          mazda: 4129.77
          smart: 3596.4
        citroen: 3796.26
         toyota: 5167.09
        hyundai: 5411.08


Top 5 brands are having distinct price gap in them. While Audi, BMW, Mercedes are most expensive, Ford and Opel are less expensive. Volkswagen lies between most and less expensive brands.

With approximately 10% of total vehicles, **Audi** has the most expensive - average $9380 - range of cars in the concerned used vehicles' category.

**volkswagen** with the highest number of vehicles in the concerned category, has average price - ~$5400 - relatively on the lower side of top brand's pricing scale.

In [35]:
# Lets calculate mean power_ps for the same brands we 
# calculated mean prices for.
brand_power_ps = OrderedDict()
for brnd in top_brands:
    mean_power = autos.loc[autos['brand']==brnd,'power_ps'].mean()
    brand_power_ps[brnd] = mean_power

### Lets put brands' mean prices and mean power_ps to comparison

In [36]:
# Before let us construct a dataframe of top_brands,
# mean prices and their mean_power_ps
# bmp:brand mean price
mean_prices = pd.Series(data=brand_pricing)
mean_powers = pd.Series(data=brand_power_ps)
brand_comp = pd.DataFrame(data={'mean_prices':mean_prices,
                         'mean_power_ps':mean_powers})

In [37]:
brand_comp.iloc[brand_comp['mean_prices'].argsort(),[1,0]]

,mean_prices,mean_power_ps
renault,2496.940394,81.516735
fiat,2836.873631,70.401011
opel,3006.000201,96.412274
peugeot,3113.860549,92.346821
smart,3596.402736,65.370821
ford,3779.269076,101.805684
citroen,3796.262673,88.761905
mazda,4129.774788,109.379603
seat,4433.419622,98.349882
nissan,4756.659634,96.063291


Just as expected, brands having expensive cars tend to be on higher side of the power_ps scale.

Top 3 most expensive brands, Audi, Mercedes and BMW are having average power_ps range to be distinctively higher - above 150 PS - than the rest. BMW being the highest average powered vehicle brand. These are followed by Skoda and others in the range of 100 PS.

For the given price range, Ford makes for the best budget brand in range of 100 PS powered engines.

In [38]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,46350,46350,46350,46350,46350.000000,46350,43798,46350.000000,44377,46350.000000,44220,46350.000000,46350.000000,43180,46350,38254,46350,46350.0,46350.000000,46350
unique,34,35514,2,1,NaN,2,8,NaN,2,NaN,244,NaN,NaN,7,40,2,74,NaN,NaN,34
top,2016-04-03,BMW_316i,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03,NaN,NaN,2016-04-06
freq,1798,75,46349,46350,NaN,23875,12558,NaN,34546,NaN,3684,NaN,NaN,28380,9799,33772,1809,NaN,NaN,10359
mean,NaN,NaN,NaN,NaN,6020.173139,NaN,NaN,2002.943797,NaN,118.387789,NaN,125736.569579,5.847141,NaN,NaN,NaN,NaN,0.0,51111.874304,NaN
std,NaN,NaN,NaN,NaN,9196.811390,NaN,NaN,7.101102,NaN,185.413457,NaN,39605.572085,3.661159,NaN,NaN,NaN,NaN,0.0,25745.938577,NaN
min,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,1927.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1250.000000,NaN,NaN,1999.000000,NaN,75.000000,NaN,100000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30853.000000,NaN
50%,NaN,NaN,NaN,NaN,3150.000000,NaN,NaN,2003.000000,NaN,109.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49828.000000,NaN
75%,NaN,NaN,NaN,NaN,7581.250000,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71739.000000,NaN


### Let's translate and replace non-English words by their English counterparts

#### 1.  Vehicle Type

In [39]:
type_mapping = {'limousine': 'limousine',
                'kleinwagen': 'compact',
                'kombi': 'minibus',
                'bus': 'bus',
                'cabrio': 'convertible',
                'coupe': 'coupe',
                'suv': 'suv',
                'andere': 'other'}
autos['vehicle_type'] = autos['vehicle_type'].map(type_mapping, na_action='ignore')
autos.vehicle_type.value_counts(dropna=False)

limousine      12558
compact        10497
minibus         8906
bus             4023
convertible     3008
NaN             2552
coupe           2456
suv             1962
other            388
Name: vehicle_type, dtype: int64

#### 2. Gearbox

In [40]:
gearbox_mapping = {'manuell': 'manual',
                   'automatik': 'automatic'}
autos['gearbox'] = autos['gearbox'].map(gearbox_mapping, na_action='ignore')

**3. Fuel Type**

In [41]:
fuel_type_mapping = {'benzin': 'benzene',
                     'diesel': 'diesel',
                     'lpg' : 'lpg',
                     'cng' : 'cng',
                     'hybrid': 'hybrid',
                     'elektro': 'electric',
                     'andere': 'other'}
autos['fuel_type'] = autos['fuel_type'].map(fuel_type_mapping, na_action='ignore')

**4. Unrepaired damage**

In [42]:
damage_mapping = {'nein': 'no',
                  'ja': 'yes'}
autos['unrepaired_damage'] = autos['unrepaired_damage'].map(damage_mapping, na_action='ignore')

### Let us change dates to numeric format: 
Convert dates from string format '2016-04-03' to ints for example: 20160404

For doing that, we just need to remove the hyphen symbols and then convert the remaining string to int dtype.

There are 5 date columns, let us loop over each one of them converting to numberic format.

I'll be following the workflow of converting non-numeric data into numeric format, which is as below:
- Step 1 : Explore the data
- Step 2 : Find any patterns
- Step 3 : Remove non-digit characters
- Step 4 : Convert (type cast) the column to numeric dtype.
- Step 5 : Rename column if required.

But, first, let's see if there are any NaN's in those columns, because nan's won't allows us to convert date columns to int dtype.

In [43]:
autos[dates_col].isnull().sum()    

date_crawled    0
ad_created      0
last_seen       0
dtype: int64

It seems there are no null values in either of those columns, which means, we're good to go with the dtype conversion process.

In [44]:
for col in dates_col:
    autos[col] = autos[col].str.replace('-','').astype(int)

# Let's check if it worked
autos[dates_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46350 entries, 0 to 49999
Data columns (total 3 columns):
date_crawled    46350 non-null int64
ad_created      46350 non-null int64
last_seen       46350 non-null int64
dtypes: int64(3)
memory usage: 1.4 MB


**GOOD.** We've successfully converted dates to int dtype.

### Let us take a closer look at 'name' column:

In [53]:
# Let's see if the name has 'automatic' in it and gearbox has missing value
pd.set_option('display.max_colwidth',-1)
autos.sample(10)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
30455,20160319,Opel_Vectra_Diesel,privat,Angebot,600.0,control,limousine,1998,manual,101,vectra,150000.0,1,diesel,opel,NaN,20160319,0,47608,20160328
41498,20160307,Mitsubishi_L200_4x4_Magnum_2._Hand_/_LKW_Zulassung_/_Klima_/_99TK,privat,Angebot,7750.0,test,suv,2001,manual,99,pajero,100000.0,12,diesel,mitsubishi,no,20160307,0,52525,20160310
33024,20160314,Bmw_318Ci_Frische_Tuev_E46,privat,Angebot,2399.0,control,limousine,2001,manual,143,3er,150000.0,1,benzene,bmw,no,20160314,0,59759,20160407
22931,20160309,Renault_Laguna_Grandtour_2.2_dCi_Initiale,privat,Angebot,2200.0,control,minibus,2003,automatic,150,laguna,150000.0,5,diesel,renault,no,20160309,0,58675,20160311
8357,20160316,Opel_Astra_mit_Tuev,privat,Angebot,749.0,test,minibus,1999,manual,101,astra,150000.0,5,benzene,opel,NaN,20160316,0,97941,20160323
10430,20160315,BMW_316i,privat,Angebot,1199.0,control,limousine,1996,manual,102,3er,150000.0,6,benzene,bmw,no,20160315,0,27753,20160315
11116,20160401,Ford_mondeo_mit_TÜV,privat,Angebot,300.0,test,limousine,1996,manual,112,mondeo,150000.0,4,NaN,ford,no,20160401,0,15234,20160407
34547,20160322,Opel_Combo_1.6,privat,Angebot,1390.0,test,minibus,2004,manual,87,combo,150000.0,8,benzene,opel,no,20160322,0,63454,20160406
31457,20160330,Peugeot_206_AC_1_6l_16V_Panoramadach_+_Alu,privat,Angebot,1950.0,test,compact,2001,manual,109,2_reihe,150000.0,7,NaN,peugeot,NaN,20160330,0,44269,20160407
24872,20160315,VW_T5_2_5_AXD_Motor,privat,Angebot,2500.0,test,bus,2005,automatic,130,transporter,150000.0,10,diesel,volkswagen,NaN,20160315,0,63322,20160315


### Let us find the most common brand-model combinations

In [78]:
brands = autos['brand'].value_counts().index
cbm_combination = OrderedDict()

for brnd in brands:
    # Model frequency for brand=brnd
    model_freq = autos.loc[autos['brand']==brnd, 'model'].value_counts()
    if model_freq.count():
        common_model = model_freq.index[0]
    else:
        common_model = np.nan
    cbm_combination[brnd] = common_model

cbm_combination = pd.Series(cbm_combination)
print(cbm_combination)

volkswagen        golf      
bmw               3er       
opel              corsa     
mercedes_benz     c_klasse  
audi              a4        
ford              focus     
renault           twingo    
peugeot           2_reihe   
fiat              punto     
seat              ibiza     
skoda             octavia   
nissan            micra     
mazda             3_reihe   
smart             fortwo    
citroen           andere    
toyota            yaris     
hyundai           i_reihe   
sonstige_autos    NaN       
volvo             v70       
mini              cooper    
mitsubishi        colt      
honda             civic     
kia               andere    
alfa_romeo        156       
porsche           911       
suzuki            andere    
chevrolet         andere    
chrysler          andere    
dacia             sandero   
daihatsu          cuore     
jeep              grand     
land_rover        freelander
subaru            legacy    
saab              andere    
jaguar        

### Let us find how damage affects the average car prices:

Here, we will try to calculate the average price of damaged cars vs the average price of non-damaged cars.

In [80]:
damaged_avg_price = autos.loc[autos['unrepaired_damage']=='yes', 'price'].mean()
non_damaged_avg_price = autos.loc[autos['unrepaired_damage']=='no','price'].mean()
difference = non_damaged_avg_price - damaged_avg_price
print("The difference between avg prices is = {:.2f}".format(difference))

The difference between avg prices is = 4907.56


The difference seems to be too big compared to actual car prices. But there are some downsides that could be affecting the approach we took in finding the difference:
    1. Expensive cars are more likely to be handled with much care and so they tend to have no damage. The point is, cars with no-damage are more likely to be expensive, causing their mean price on higher side compared to that of damaged-less_expensive cars.

Possibly, due to this reason, there's a huge difference between the mean prices of damaged cars vs non-damaged cars.